<a href="https://colab.research.google.com/github/jawad786-lab/reddit-flare-detection/blob/master/flare_train_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install praw

     |████████████████████████████████| 143kB 2.8MB/s 
     |████████████████████████████████| 204kB 8.8MB/s 


In [0]:
pip install gensim nltk

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import praw
import pandas as pd
import datetime as dt
import logging
import numpy as np
from numpy import random
import gensim
import nltk
nltk.download('all')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report
import json

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

In [0]:
flairs = ["AskIndia", "Non-Political", "[R]eddiquette", "Scheduled", "Photography", "Science/Technology", "Politics", "Business/Finance", "Policy/Economy", "Sports", "Food", "AMA"]
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

In [0]:
def get_date(created):
    return dt.datetime.fromtimestamp(created)

def string_form(value):
    return str(value)

def clean_text(text):
   
    text = BeautifulSoup(text, "lxml").text
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = BAD_SYMBOLS_RE.sub('', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text

In [0]:
reddit = praw.Reddit(client_id='2molZnY9WZK4yg', client_secret='FLmiHcdZs8J1pGKwzMRdUQLDhSM', user_agent='Reddit Webs Scraping')

subreddit = reddit.subreddit('india')
topics_dict = {"flair":[], "title":[], "score":[], "id":[], "url":[], "comms_num": [], "created": [], "body":[], "author":[], "comments":[]}

for flair in flairs:
  
  get_subreddits = subreddit.search(flair, limit=100)
  
  for submission in get_subreddits:
    
    topics_dict["flair"].append(flair)
    topics_dict["title"].append(submission.title)
    topics_dict["score"].append(submission.score)
    topics_dict["id"].append(submission.id)
    topics_dict["url"].append(submission.url)
    topics_dict["comms_num"].append(submission.num_comments)
    topics_dict["created"].append(submission.created)
    topics_dict["body"].append(submission.selftext)
    topics_dict["author"].append(submission.author)
    
    submission.comments.replace_more(limit=10)
    comment = ''
    for top_level_comment in submission.comments:
      comment = comment + ' ' + top_level_comment.body
    topics_dict["comments"].append(comment)
    
topics_data = pd.DataFrame(topics_dict)
_timestamp = topics_data["created"].apply(get_date)
topics_data = topics_data.assign(timestamp = _timestamp)
del topics_data['created']
topics_data.to_csv('reddit-india-data.csv', index=False)

In [0]:
topics_data = pd.read_csv('reddit-india-data.csv')
topics_data.head(10)

,flair,title,score,id,url,comms_num,body,author,comments,timestamp
0,AskIndia,4 days ago we had pending orders of 100 millio...,94,fwjdqr,https://www.reddit.com/r/india/comments/fwjdqr...,6,> We are getting frantic calls from our pharma...,india_ko_vanakkam,"Modi has Stockholm syndrome To be fair, the e...",2020-04-07 20:07:04
1,AskIndia,Randians who were big time users of dating app...,22,fizkkk,https://www.reddit.com/r/india/comments/fizkkk...,19,I'd my own stint with these apps(a couple of m...,__knockknockturnal__,Someone matched with me just to tell me that ...,2020-03-15 18:48:06
2,AskIndia,What does r/India thinks about the Flat Earthers?,7,f25vx0,https://www.reddit.com/r/india/comments/f25vx0...,31,"I encountered a Foreigner in IG who says "" Rou...",Dev1003,I haven't found a Indian yet who believes ear...,2020-02-11 17:10:55
3,AskIndia,People who left their 9 to 5 jobs to pursue a ...,43,dtvliq,https://www.reddit.com/r/india/comments/dtvliq...,34,Couldn't add AskIndia flair from the mobile br...,c0mrade34,"An Engineer, doing advertisement shoots since...",2019-11-09 20:57:35
4,AskIndia,Need feedback for Insurance Policy that I took...,1,1s57oi,https://www.reddit.com/r/india/comments/1s57oi...,1,**Re-posting here because of lack of activity ...,dhavalcoholic,"Dear Policy Holder(Dhavalcoholic),\n \nWe req...",2013-12-05 14:30:23
5,AskIndia,Somebody want to kill my full family what to do?,95,b7pvwt,https://www.reddit.com/r/india/comments/b7pvwt...,24,"It's now 24hrs, But local police station is no...",amitkumarthakur,"Calm down.\nGo to the SP office of your town,...",2019-04-01 01:00:35
6,AskIndia,Ambassador of India takes back my newly issued...,11,bdfid1,https://www.reddit.com/r/india/comments/bdfid1...,27,"Hello /AskIndia! First time poster, long time...",FrustratedOCIHopeful,"Honestly, she and her supervisor behaved *exa...",2019-04-15 20:41:42
7,AskIndia,"Randians, what are you too afraid to ask?",19,cu1xn4,https://www.reddit.com/r/india/comments/cu1xn4...,22,r/TooAfraidToAsk India edition,aloo_vs_bhaloo,How does Modi control his sex desires? Or if ...,2019-08-23 03:00:59
8,AskIndia,Toothpaste alternative?,11,dfljx0,https://www.reddit.com/r/india/comments/dfljx0...,8,Hello India. I'm currently trying to find a to...,Hyhyhyhuh,Chew neem stick in desi style [https://en.wik...,2019-10-10 02:54:21
9,AskIndia,"[AskIndia] Cingari, Cengar or Tzengar?",0,18ntue,https://www.reddit.com/r/india/comments/18ntue...,0,"Hello,\n\nI submitted this to /r/rAskIndia a w...",multubunu,NaN,2013-02-17 07:08:21


In [0]:
topics_data['title'] = topics_data['title'].apply(string_form)
topics_data['body'] = topics_data['body'].apply(string_form)
topics_data['comments'] = topics_data['comments'].apply(string_form)
topics_data['title'] = topics_data['title'].apply(clean_text)
topics_data['body'] = topics_data['body'].apply(clean_text)
topics_data['comments'] = topics_data['comments'].apply(clean_text)
feature_combine = topics_data["title"] + topics_data["comments"] + topics_data["url"]
topics_data = topics_data.assign(feature_combine = feature_combine)

In [0]:
def nb_classifier(X_train, X_test, y_train, y_test):
  
  from sklearn.naive_bayes import MultinomialNB


  nb = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('clf', MultinomialNB()),
                ])
  nb.fit(X_train, y_train)

  y_pred = nb.predict(X_test)

  print('accuracy %s' % accuracy_score(y_pred, y_test))
  print(classification_report(y_test, y_pred,target_names=flairs))

In [0]:
def linear_svm(X_train, X_test, y_train, y_test):
  
  from sklearn.linear_model import SGDClassifier

  sgd = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
                 ])
  sgd.fit(X_train, y_train)

  y_pred = sgd.predict(X_test)

  print('accuracy %s' % accuracy_score(y_pred, y_test))
  print(classification_report(y_test, y_pred,target_names=flairs))

In [0]:
def logisticreg(X_train, X_test, y_train, y_test):

  from sklearn.linear_model import LogisticRegression

  logreg = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', LogisticRegression(n_jobs=1, C=1e5)),
                 ])
  logreg.fit(X_train, y_train)

  y_pred = logreg.predict(X_test)

  print('accuracy %s' % accuracy_score(y_pred, y_test))
  print(classification_report(y_test, y_pred,target_names=flairs))

In [0]:
def train_test(X,y):
 
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

  print("Results of Naive Bayes Classifier")
  nb_classifier(X_train, X_test, y_train, y_test)
  print("Results of Linear Support Vector Machine")
  linear_svm(X_train, X_test, y_train, y_test)
  print("Results of Logistic Regression")
  logisticreg(X_train, X_test, y_train, y_test)

In [0]:
cat = topics_data.flair

V = topics_data.feature_combine
W = topics_data.comments
X = topics_data.title
Y = topics_data.body
Z = topics_data.url

print("Flair Detection using Title as Feature")
#train_test(X,cat)
print("Flair Detection using Body as Feature")
#train_test(Y,cat)
print("Flair Detection using URL as Feature")
#train_test(Z,cat)
print("Flair Detection using Comments as Feature")
train_test(V,cat)

Flair Detection using Title as Feature
Flair Detection using Body as Feature
Flair Detection using URL as Feature
Flair Detection using Comments as Feature
Results of Naive Bayes Classifier
accuracy 0.4583333333333333
                    precision    recall  f1-score   support

          AskIndia       0.17      0.93      0.29        28
     Non-Political       0.60      0.09      0.15        34
     [R]eddiquette       0.30      0.52      0.38        25
         Scheduled       0.77      0.89      0.83        27
       Photography       0.60      0.24      0.34        25
Science/Technology       0.96      0.71      0.81        34
          Politics       0.83      0.29      0.43        34
  Business/Finance       0.62      0.14      0.23        36
    Policy/Economy       0.89      0.26      0.40        31
            Sports       0.86      0.70      0.78        27
              Food       0.88      0.50      0.64        30
               AMA       1.00      0.20      0.33         5



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
